# Gradients, a work in progress...
---
This notebook will in detail go through the most essential operators and their forward/backward pass respectively.

### Add
$f(x, y) = x + y$<br>
$\frac{\partial f}{\partial x} = 1$<br>
$\frac{\partial f}{\partial y} = 1$

### Sub
$f(x, y) = x - y$<br>
$\frac{\partial f}{\partial x} = 1$<br>
$\frac{\partial f}{\partial y} = -1$

### Mean
Assuming that we take the mean over the entire ndarray $x$,<br>
$f(x) = \frac{1}{N}\sum_i x_i$<br>
$\frac{\partial f}{\partial x} = \frac{1}{N}(1_1 + \cdots + 1_N)$<br>

Now, lets assume that $x$ is n-dimensional, with varying number of elements for each dimension, and that we want to take the mean over some specifix axis, lets denote the product of the input shape and output shape as $\alpha$ and $\beta$ respectively, then<br>
$f(x) = \frac{\alpha}{\beta} \sum_i^\alpha x_i$<br>
$\frac{\partial f}{\partial x} = \frac{\alpha}{\beta}(1_1 + \cdots + 1_\alpha)$

### Sum
Once again, lets assume that we summarize the full ndarray $x$,<br>
$f(x) = \sum_i x_i$<br>
$\frac{\partial f}{\partial x} = 1 + \cdots + 1$

### Log
$f(x) = log(x)$<br>
$\frac{\partial f}{\partial x} = \frac{1}{x}$

### Exp
$f(x) = e^x$<br>
$\frac{\partial f}{\partial x} = e^x$
